In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
from gensim.models import Word2Vec
from nltk.corpus import stopwords
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,balanced_accuracy_score,roc_auc_score
import os.path
from nltk.stem import SnowballStemmer, PorterStemmer
import nltk
import re
import json
import os.path
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import BaggingClassifier
from sklearn.svm import SVC
from gensim.models import KeyedVectors
import gensim.downloader
import time

In [2]:
data = pd.read_csv("drive/MyDrive/toxic_classification.csv")
df_0 = data[data['target'] == 0].iloc[:20000, :]
df_1 = data[data['target'] == 1].iloc[:20000, :]
df_small = pd.concat([df_0, df_1])
nltk.download("popular")

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/names.zip.
[nltk_data]    | Downloading package shakespeare to /root/nltk_data...
[nlt

True

In [3]:
# Load the data
y = df_small['target'].astype('int').to_numpy()

stemmer = SnowballStemmer("english")
stop_words = set(stopwords.words('english'))
def preprocess(text_string):
    text_string = text_string.lower() 
    text_string = re.sub('[^A-Za-z0-9]+', ' ', text_string)     
    x = text_string.split()
    new_text = []    
    for word in x:
        if word not in stop_words:
            new_text.append(stemmer.stem(word))          
    text_string = ' '.join(new_text)
    return text_string
if os.path.isfile("drive/MyDrive/small_preprocessX") !=True:
    processed_X = [preprocess(i) for i in data.comment_text]
    with open("drive/MyDrive/small_preprocessX", "w") as fp:
        json.dump(processed_X, fp)
else:
    with open('drive/MyDrive/small_preprocessX', "r") as fp:
        processed_X = json.load(fp)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(processed_X,y, test_size=0.2, random_state=0)
# load the Word2Vec model
from gensim.models import KeyedVectors
filename = 'word2vec-google-news-300'
w2v_model = gensim.downloader.load(filename)

[=================================================-] 99.9% 1660.8/1662.8MB downloaded


In [5]:
# Train the Word2Vec model
sentences = [sentence.split() for sentence in X_train]
time.perf_counter()
w2v_mode2 = Word2Vec(sentences, vector_size=300,sg=1, window=5, min_count=5) 
time.perf_counter()

691.947090952

In [6]:
#w2v_model.save("drive/MyDrive/small_w2v_size100_window5_min5_skipgram.model")
#w2v_mode2 =Word2Vec.load("drive/MyDrive/small_w2v_size100_window5_min5_skipgram.model")
#w2v_model = Word2Vec.load("drive/MyDrive/small_w2v_size100_window5_min5_workers4.model")


In [7]:
a=time.perf_counter()
tf_idf_vectorizer = TfidfVectorizer(token_pattern= r'\S+')
X = tf_idf_vectorizer.fit_transform(processed_X)
b=time.perf_counter()
print("Time: "+ str(b-a))

Time: 1.0742212159999553


In [8]:
X_array = X.toarray()

In [9]:
words_set = tf_idf_vectorizer.get_feature_names_out()
df_tf_idf = pd.DataFrame(X_array, columns = words_set)

In [10]:
# Vectorize data
def vectorize(sentence,modelwv):
    words = sentence.split()
    words_vecs = [modelwv[word] for word in words if word in modelwv]
    if len(words_vecs) == 0:
        return np.zeros(300)
    words_vecs = np.array(words_vecs)
    return words_vecs.mean(axis=0)

In [11]:
def vectorize_weighted(i,sentence,modelwv):
    words_vecs = [modelwv[word]*df_tf_idf[word].loc[i] for word in sentence if word in modelwv]
    if len(words_vecs) == 0:
        return np.zeros(300)
    words_vecs = np.array(words_vecs)
    return words_vecs.mean(axis=0)

In [12]:
#vectorizer = TfidfVectorizer()
#X_tfidf = vectorizer.fit_transform(processed_X)

In [13]:
#X_train_s = np.array([vectorize(sentence,w2v_mode2.wv) for sentence in X_train])
#X_test_s = np.array([vectorize(sentence,w2v_mode2.wv) for sentence in X_test])
X_train_s = np.array([vectorize(sentence,w2v_model) for sentence in X_train])
X_test_s = np.array([vectorize(sentence,w2v_model) for sentence in X_test])

In [14]:
sentences = [sentence.split() for sentence in processed_X]
#X_w = np.array([vectorize_weighted(i,sentence,w2v_mode2.wv) for i,sentence in enumerate(sentences)])
X_w = np.array([vectorize_weighted(i,sentence,w2v_model) for i,sentence in enumerate(sentences)])
X_train_sw, X_test_sw, y_train, y_test = train_test_split(X_w,y, test_size=0.2, random_state=0)

In [15]:
len(X_train_s[1])

300

In [16]:
a=time.perf_counter()
bagging_SVM_weighted = BaggingClassifier(SVC(kernel= 'linear', random_state=1, C=0.1,verbose=True),n_estimators=5,max_samples=0.99,verbose=True)
bagging_SVM_weighted.fit(X_train_sw, y_train)
b=time.perf_counter()
print("Time: "+ str(b-a))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[LibSVM][LibSVM][LibSVM][LibSVM][LibSVM]Time: 614.5006338830001


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 10.2min finished


In [17]:
a=time.perf_counter()
bagging_SVM_s = BaggingClassifier(SVC(kernel= 'linear', random_state=1, C=0.1,verbose=True),n_estimators=5,max_samples=0.99,verbose=True)
bagging_SVM_s.fit(X_train_s, y_train)
b=time.perf_counter()
print("Time: "+ str(b-a))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[LibSVM][LibSVM][LibSVM][LibSVM][LibSVM]Time: 482.24718326899983


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  8.0min finished


In [18]:
a=time.perf_counter()
y_bg_svm = bagging_SVM_weighted.predict(X_test_sw)
b=time.perf_counter()
print("Time: "+ str(b-a))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Time: 152.309637029


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.5min finished


In [19]:
a=time.perf_counter()
y_bg_svm_s = bagging_SVM_s.predict(X_test_s)
b=time.perf_counter()
print("Time: " +str(b-a))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Time: 105.120168939


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.8min finished


In [20]:
print('Accuracy: %.3f' % accuracy_score(y_test, y_bg_svm))
#print('Balanced Accuracy: %.3f' %balanced_accuracy_score(y_test, y_bg_svm))
print('Precision:', precision_score(y_test, y_bg_svm))
print('Recall:', recall_score(y_test, y_bg_svm))
print('F1: %.3f' % f1_score(y_test, y_bg_svm))
#print('Roc_auc: %.3f' % roc_auc_score(y_test, y_bg_svm))

Accuracy: 0.700
Precision: 0.8045801526717558
Recall: 0.5275275275275275
F1: 0.637


In [21]:
print('Accuracy: %.3f' % accuracy_score(y_test, y_bg_svm_s))
#print('Balanced Accuracy: %.3f' %balanced_accuracy_score(y_test, y_bg_svm_s))
print('Precision:', precision_score(y_test, y_bg_svm_s))
print('Recall:', recall_score(y_test, y_bg_svm_s))
print('F1: %.3f' % f1_score(y_test, y_bg_svm_s))
#print('Roc_auc: %.3f' % roc_auc_score(y_test, y_bg_svm_s))

Accuracy: 0.792
Precision: 0.8126338329764454
Recall: 0.7597597597597597
F1: 0.785


In [22]:
#pipe = make_pipeline(StandardScaler(with_mean=True), LogisticRegression())
#bagging_LR = BaggingClassifier(pipe,n_estimators=5,max_samples=0.99,verbose=True)
#bagging_LR.fit(X_train_sw, y_train)
#y_bg_LR = bagging_LR.predict(X_test_sw)

#bagging_LR_s = BaggingClassifier(pipe,n_estimators=5,max_samples=0.99,verbose=True)
#bagging_LR_s.fit(X_train_s, y_train)
#y_bg_LR_s = bagging_LR_s.predict(X_test_s)

In [23]:
#print('Accuracy: %.3f' % accuracy_score(y_test, y_bg_LR))
#print('Balanced Accuracy: %.3f' %balanced_accuracy_score(y_test, y_bg_LR))
###print('Precision:', precision_score(y_test, y_bg_LR))
#print('Recall:', recall_score(y_test, y_bg_LR))
#print('F1: %.3f' % f1_score(y_test, y_bg_LR))
#print('Roc_auc: %.3f' % roc_auc_score(y_test, y_bg_LR))

In [24]:
#print('Accuracy: %.3f' % accuracy_score(y_test, y_bg_LR_s))
#print('Balanced Accuracy: %.3f' %balanced_accuracy_score(y_test, y_bg_LR_s))
#print('Precision:', precision_score(y_test, y_bg_LR_s))
#print('Recall:', recall_score(y_test, y_bg_LR_s))
#print('F1: %.3f' % f1_score(y_test, y_bg_LR_s))
#print('Roc_auc: %.3f' % roc_auc_score(y_test, y_bg_LR_s))

In [25]:
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
a=time.perf_counter()
clf = MLPClassifier(random_state=1,hidden_layer_sizes=(250,25), max_iter=200,early_stopping = True,verbose = True).fit(X_train_sw, y_train)
b=time.perf_counter()
print("Time: " +str(b-a))

Iteration 1, loss = 0.58264185
Validation score: 0.761563
Iteration 2, loss = 0.49677459
Validation score: 0.760625
Iteration 3, loss = 0.47916996
Validation score: 0.774687
Iteration 4, loss = 0.46521721
Validation score: 0.775000
Iteration 5, loss = 0.45384114
Validation score: 0.777813
Iteration 6, loss = 0.44285388
Validation score: 0.781563
Iteration 7, loss = 0.43454441
Validation score: 0.780937
Iteration 8, loss = 0.42378701
Validation score: 0.780000
Iteration 9, loss = 0.41430330
Validation score: 0.785000
Iteration 10, loss = 0.40406948
Validation score: 0.785312
Iteration 11, loss = 0.39420665
Validation score: 0.780937
Iteration 12, loss = 0.38600532
Validation score: 0.782813
Iteration 13, loss = 0.37683821
Validation score: 0.785312
Iteration 14, loss = 0.36676190
Validation score: 0.782500
Iteration 15, loss = 0.35586067
Validation score: 0.785625
Iteration 16, loss = 0.34743386
Validation score: 0.790000
Iteration 17, loss = 0.33847758
Validation score: 0.787188
Iterat

In [26]:
a=time.perf_counter()
clf_s = MLPClassifier(random_state=1,hidden_layer_sizes=(250,25), max_iter=200,early_stopping = True,verbose = True).fit(X_train_s, y_train)
b=time.perf_counter()
print("Time: "+ str(b-a))

Iteration 1, loss = 0.51083528
Validation score: 0.794687
Iteration 2, loss = 0.43726661
Validation score: 0.800312
Iteration 3, loss = 0.42042763
Validation score: 0.804688
Iteration 4, loss = 0.40675433
Validation score: 0.800312
Iteration 5, loss = 0.39233983
Validation score: 0.806250
Iteration 6, loss = 0.37786431
Validation score: 0.803438
Iteration 7, loss = 0.36706024
Validation score: 0.809063
Iteration 8, loss = 0.35267126
Validation score: 0.811562
Iteration 9, loss = 0.34066092
Validation score: 0.816562
Iteration 10, loss = 0.32423725
Validation score: 0.811562
Iteration 11, loss = 0.31038351
Validation score: 0.815625
Iteration 12, loss = 0.29897692
Validation score: 0.806562
Iteration 13, loss = 0.28493885
Validation score: 0.805937
Iteration 14, loss = 0.27052472
Validation score: 0.810937
Iteration 15, loss = 0.25850772
Validation score: 0.814063
Iteration 16, loss = 0.24703755
Validation score: 0.810625
Iteration 17, loss = 0.23395321
Validation score: 0.802813
Iterat

In [27]:
y_mlp = clf.predict(X_test_sw)
print('Accuracy: %.3f' % accuracy_score(y_test, y_mlp))
#print('Balanced Accuracy: %.3f' %balanced_accuracy_score(y_test, y_mlp))
print('Precision:', precision_score(y_test, y_mlp))
print('Recall:', recall_score(y_test, y_mlp))
print('F1: %.3f' % f1_score(y_test, y_mlp))
#print('Roc_auc: %.3f' % roc_auc_score(y_test, y_mlp))

Accuracy: 0.791
Precision: 0.8016636340005199
Recall: 0.7717717717717718
F1: 0.786


In [28]:
y_mlp_s = clf_s.predict(X_test_s)
print('Accuracy: %.3f' % accuracy_score(y_test, y_mlp_s))
#print('Balanced Accuracy: %.3f' %balanced_accuracy_score(y_test, y_mlp_s))
print('Precision:', precision_score(y_test, y_mlp_s))
print('Recall:', recall_score(y_test, y_mlp_s))
print('F1: %.3f' % f1_score(y_test, y_mlp_s))
#print('Roc_auc: %.3f' % roc_auc_score(y_test, y_mlp_s))

Accuracy: 0.817
Precision: 0.8060928433268859
Recall: 0.8343343343343343
F1: 0.820
